In [ ]:
!pip install pandas
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu cromadb
!pip install -qU kiwipiepy konlpy langchain-teddynote tiktoken

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "dacon"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [ ]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("dacon")

In [ ]:
# for Retrieval
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever

# for llm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# for document loader
from langchain_community.document_loaders import PyMuPDFLoader

# for chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

# for embedding
from langchain_huggingface import HuggingFaceEmbeddings

# for vectorDB
from langchain_community.vectorstores import Chroma
from langchain.vectorstores import FAISS

# for chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain.schema import Document

# for huggingface
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from transformers import pipeline

# Parent Document Retriever
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever


# for data
import zipfile
import pandas as pd
import unicodedata
import fitz

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"문서 {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
def format_docs(docs):
    unique_contents = set(doc.page_content for doc in docs)
    return "\n\n".join(unique_contents)

In [ ]:
# 쿼리 처리
df = pd.read_csv("test.csv")

# Question 열의 텍스트를 리스트로 저장
questions = df['Question'].tolist()

In [ ]:
# query test
query_test = questions[3]
query_test

In [ ]:
### Document loader ###
pdfs = [
    "./test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf",
    "./test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf",
    "./test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf",
    "./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf",
    "./test_source/국토교통부_행복주택출자.pdf",
    "./test_source/보건복지부_노인장기요양보험 사업운영.pdf",
    "./test_source/보건복지부_부모급여(영아수당) 지원.pdf",
    "./test_source/산업통상자원부_에너지바우처.pdf",
    "./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf"
]

docs = [PyMuPDFLoader(pdf).load() for pdf in pdfs]
docs_list = [item for sublist in docs for item in sublist ]

# 총 데이터의 길이 출력
print("총 데이터 수:", len(docs_list))

In [ ]:
### CSV loader ###
from langchain_community.document_loaders.csv_loader import CSVLoader
csvs = [
    "./test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.csv",
    "./test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.csv",
    "./test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.csv",
    "./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.csv",
    "./test_source/국토교통부_행복주택출자.csv",
    "./test_source/보건복지부_노인장기요양보험 사업운영.csv",
    "./test_source/보건복지부_부모급여(영아수당) 지원.csv",
    "./test_source/산업통상자원부_에너지바우처.csv",
    "./test_source/중소벤처기업부_혁신창업사업화자금(융자).csv"
]

# 데이터를 저장할 리스트
csv_data = []

# 각 CSV 파일을 로드하고 데이터를 리스트에 저장
for csv_file in csvs:
    loader = CSVLoader(file_path=csv_file, encoding='utf-8')
    data = loader.load()
    csv_data.extend(data)  # 각 파일의 데이터를 리스트에 추가

# 총 데이터의 길이 출력
print("총 데이터 수:", len(csv_data))

In [ ]:
# PDF 데이터와 CSV 데이터를 결합
all_list = docs_list + csv_data
print("총 데이터 수:", len(all_list))

In [ ]:
### Embedding ###
model_kwargs = {'device': 'cpu'} # gpu -> 'cuda', cpu -> 'cpu'

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs=model_kwargs,
    encode_kwargs=model_kwargs,
    show_progress=True
)

In [ ]:
### Chatmodel 불러오기 ###
# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "meta-llama/Meta-Llama-3.1-405B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype = "bfloat16",
    device_map = 'auto',
    trust_remote_code=True
)

In [ ]:
# 파이프라인 선언
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.0, #모델의 창의성
    return_full_text=False,
    max_new_tokens=1024, #최대 생성 토큰 수
    #repetition_penalty=2.0
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
### csv + pdf ###
child_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024, chunk_overlap=50
)
# DB를 생성합니다.
vectorstore = Chroma(
    collection_name="full_documents", 
    embedding_function=embeddings
)
store = InMemoryStore()

# Retriever 를 생성합니다.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

retriever.add_documents(all_list, ids=None, add_to_docstore=True)

In [ ]:
### 1. 그냥 retriever ###
child_retriever = vectorstore.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 5, "fetch_k": 10}
)

In [ ]:
### 2. (kiwi retriever ###
from langchain_teddynote.retrievers import KiwiBM25Retriever
kiwi = KiwiBM25Retriever.from_documents(all_list)
kiwi.k = 5
print(query_test)
pretty_print_docs(kiwi.invoke(query_test))

In [ ]:
### 2. Ensemble ###
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
                    retrievers = [kiwi, child_retriever],
                    weights = [0.2, 0.8],
                    search_type="mmr",
                    search_kwargs={'k':2})

In [ ]:
### 3. reranker + ensemble ###
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

query_test = questions[0]

# 모델 초기화 
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
#reranker = HuggingFaceCrossEncoder(model_name="Dongjin-kr/ko-reranker")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=reranker, top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=child_retriever) # base retriever를 바꿔주세요


In [ ]:
### 모든 질문에 대해 수행 ###
answers = []
for query in tqdm(questions, desc="Answer number"):
    documents = compression_retriever.invoke(query)
    
    # 문서 내용을 저장할 변수 초기화
    documents_string = ""

    for document in documents:
        documents_string += f"""
----------------------------
{document.page_content}
"""
    # Prompt
    prompt = PromptTemplate(
        template="""
            당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요. 
            문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.
            순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.
            수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.
            질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.

            답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, '답변: ', '참고: '를 절대로 출력하지 마세요. 
        
            문서:
            {document},

            질문:
            {query},
            
            주어진 질문에 대한 답변만 한 문장으로 생성한다.    
            """,
        input_variables=["document", "query"] 
    )

    print(f"Question: {query}")

    chain = (
        {"document": compression_retriever | format_docs, "query": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )

    result = chain.invoke(query)
    answers.append(result)
    print(f"Answer: {result}\n")

submission_df = pd.read_csv("sample_submission.csv")

submission_df['Answer'] = answers

# csv 이름을 바꿔주세요
submission_df.to_csv("submission.csv", index=False)